# Prompt and answer

This notebook shows how to use the API of chat-gpt in python. It contains a simple prompt and answer script. In its simplest form you send a question (called a prompt) to API at openai.com and the reply of the GPT model is returned. 
There are a few general properties to keep in mind:
- The pre-learning was performed by letting the model predict the next word. This model was trained on an enormous amount of text, mainly scraped from internet.
- It is a language model, and it performs well in many languages, but excells in English. This is probably caused by the larger amount of training material in English. 
- It sees programming languages simply as langauges. It knows of many programming languages, but is probably best in python. Again probably caused by the amount of training material.
- The model is generative, which means that it generates text with a random touch to it. Twice the same question will generally give two different answers.
- The style is kept separate from the content, so you can give style hints, like: "describe to in laymen's terms what generative deep learning models are." You can look at https://prompts.chat for inspiration.
- You will need an api-key to get access and for billing to an account.
- Every api call will cost money. At the moment roughly 0.3 cents per thousand tokens for GPT-3.5-turbo, and a token is roughly a syllable long. GPT-4 is more capable but 10 times as expensive, and not everyone get's access.

There are many things you can ask. Here are a few examples:
- Can you explain the prandtl length scale model for turbulence
- Can you give an example of how to make a gif movie with matplotlib in python
- You are a translation robot from english to french. You will receive a text in english and will reply in french. Just give the answer. "To be or not to be, that's the question."

More info can be found at:
- https://www.datacamp.com/tutorial/using-gpt-models-via-the-openai-api-in-python
- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/api-reference

In [ ]:
# installation on colab
# run this cell to install the required packages, but only once
# You may have to restart the runtime after running this cell (Runtime -> Restart runtime)
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  %pip install openai
  # getpass already installed
else:
  print('Make sure that you have installed the required packages listed in requirements.txt')

In [ ]:
#imports
import openai
import getpass
from IPython.display import display, Markdown

In [ ]:
#input your API key
openai.api_key = getpass.getpass("Enter your API key: ")

In [ ]:
# use the API to generate the answer
max_tokens = 64
response = openai.Completion.create(
    model="text-davinci-003", # this is a simpler model than gpt-3, but it's still often useful
    prompt="Translate the following English text to French: 'I love it when a plan comes together'",
    max_tokens=max_tokens
)

print(response.choices[0].text.strip())


In [ ]:
# Show complete response
print(response)

## The gtp3 and gpt4 models use a chat api

The dialog between you and gpt takes the form of a list of messages, where each adds an item to the list in turn. The three most important roles are:
- system: here you tell gpt what role to play with the input that follows, e.g. "you are weird scientist giving evasive answers"
- user: the input into the model
- assistant: the replies from gpt. If you input these for the next question, then you will provide the history of the Q&A and can also refer back to previous questions.

The inputs and outputs of the model are structures with fields and lists. In the web-api they're given in JSON format, but in python they're translated into python structures like:

``[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Hello!"}]``

In [ ]:
# List the available engines, which will depend on your API key
response = openai.Model.list()

#print(response["data"][1])
for item in response['data']:
    id = item['id']
    if "gpt" in id:
        print("- ",id)

In [ ]:
# Some settings
#model: typically use "text-davinci-003" for testing, "gpt-3.5-turbo" for production and "gpt-4" for difficult questions
model = "gpt-3.5-turbo" 
max_tokens = 100

In [ ]:
# GPT-3.5 Turbo 
# The input to this model slightly more complex than the previous model. It looks like this:
# [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Hello!"}]
messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Hello!"}]
response = openai.ChatCompletion.create( #note that we use ChatCompletion instead of Completion
    model=model,
    messages=messages,
    max_tokens=max_tokens
)

print("Full response:",response)
print("=====================================")
print("Just the actual reply:",response["choices"][0]["message"]["content"])

In [ ]:
# Simple loop to keep asking for prompts
messages = [{"role": "system", "content": "You are a helpful assistant."}] # initialize with a system message
display(Markdown("Type quit to stop ")) 
while True:
    prompt = input("Enter your prompt: ")
    if prompt == "quit":
        break
    #print("You: " + prompt)
    display(Markdown("__You:__ " + prompt))
    messages.append({"role": "user", "content": prompt}) # add the user message to the list of messages
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens
    )
    answer = response["choices"][0]["message"]
    #add the respose to the list of messages, so that it can be used as input for the next prompt
    messages.append({"role": "system", "content": answer["content"]}) 
    #print("GPT:",answer["content"])
    display(Markdown("__GPT:__ " + answer["content"]))